preproceso de datos para entrenar la red

## Datos

In [ ]:
import pandas as pd

df = pd.read_parquet("hf://datasets/wybxc/minecraft_items/data/train-00000-of-00001.parquet")
#usamos unicamente los datos del propio minecraft
df=df[df["source"]=="minecraft"]

In [7]:
df.head()

,image,label,source,prompt
7731,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,acacia boat,minecraft,"TOK style, minecraft item, acacia boat, white ..."
7732,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,acacia chest boat,minecraft,"TOK style, minecraft item, acacia chest boat, ..."
7733,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,acacia door,minecraft,"TOK style, minecraft item, acacia door, white ..."
7734,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,acacia hanging sign,minecraft,"TOK style, minecraft item, acacia hanging sign..."
7735,"{'bytes': b""\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...",acacia sign,minecraft,"TOK style, minecraft item, acacia sign, white ..."


#### preprocesado de datos:

Eliminar columnas ["source", "prompt"]

In [12]:
df= df.drop(["source", "prompt"],axis=1)

#### Guardar los datos en un parquet

In [15]:
df.to_parquet("datos/minecraft.parquet")